In [16]:
import base64
import htisec
user=base64.b64decode(b'YWNoZWNj').decode()
pwd=base64.b64decode(b'YjgyMDkxMQ==').decode()
web1=htisec.htisec_web()
txt=web1.login_1(user,pwd)

In [17]:
otp='992539'
web1.login_2(txt,otp)

1

In [29]:
import pandas  as pd
res=pd.DataFrame(index=['hold','close','total'])
hh=web1.get_orderlist()
tradelist=web1.get_tradelist(hh)
hold=web1.get_hold(hh)

point1=50
charges=67.08
c_point=round(charges/point1,1)

#hold-----------
hold_b=hold.loc[hold.trade_qty>0,'trade_qty'].sum()
hold_s=hold.loc[hold.trade_qty<0,'trade_qty'].sum()
hold_t=hold_b+hold_s
hold_sum=sum(hold['trade_qty']*hold['trade_price'])
hold_cost=round(hold_sum/hold_t if hold_t!=0 else 0,1)
hold_cost=-hold_cost if hold_t<0 else hold_cost
hold_PointT=round(hold_cost+c_point if hold_t<0 else hold_cost+c_point,1)
res.loc['hold','Bqty']=hold_b
res.loc['hold','Sqty']=hold_s
res.loc['hold','Hqty']=hold_t
res.loc['hold','Sum']=hold_sum
res.loc['hold','Cost']=hold_cost
res.loc['hold','Charges']=abs(hold_t)*charges
res.loc['hold','PointT']=hold_PointT

#close-----------
close_b=tradelist.loc[tradelist.trade_qty>0,'trade_qty'].sum()
close_s=tradelist.loc[tradelist.trade_qty<0,'trade_qty'].sum()
close_t=min(close_b,-close_s)
lots_sum=sum(tradelist['trade_qty']*tradelist['trade_price'])
c_pointT=-lots_sum-abs(hold_sum) if hold_t<0 else hold_sum-lots_sum
c_point_net=c_pointT-close_t*c_point
c_profit=c_pointT*point1
c_profit_net=c_profit-close_t*charges
res.loc['close','Bqty']=close_b
res.loc['close','Sqty']=close_s
res.loc['close','Hqty']=close_t
res.loc['close','Sum']=lots_sum
res.loc['close','Charges']=close_t*charges
res.loc['close','PointT']=c_pointT
res.loc['close','Point_NET']=c_point_net
res.loc['close','Point_AVG']=round(c_pointT/close_t-c_point,1) if close_t!=0 else 0
res.loc['close','Profit']=c_profit
res.loc['close','Profit_NET']=c_profit_net
res.loc['close','Profit_AVG']=round((c_profit-close_t*charges)/close_t,2)

#total
total_t=max(close_b,-close_s)
Hqty=close_b+close_s
total_PointT=round((lots_sum+total_t*c_point)/abs(Hqty) if Hqty!=0 else 0,1)
Point_NET=-total_PointT+hold_PointT if Hqty<0 else total_PointT-hold_PointT
res.loc['total','Bqty']=close_b
res.loc['total','Sqty']=close_s
res.loc['total','Hqty']=Hqty
res.loc['total','Sum']=lots_sum
res.loc['total','Charges']=total_t*charges
res.loc['total','Cost']=round(lots_sum/abs(Hqty)  if Hqty!=0 else 0,1)
res.loc['total','PointT']=total_PointT
res.loc['total','Point_NET']=Point_NET

rng=range(8,-7,-1)
open_lots=range(0,14,2)
cols=[ str(i)+'W' for i in rng]
trade=pd.DataFrame(columns=cols)
for ll in open_lots:
    for i in rng:   
        qty=0
        if ll==0 and Hqty!=0:
            qty=abs(Hqty)
        elif ll>0:
            qty=ll
        if qty>0:            
            winp=(i*10000/point1-c_point_net)/qty+c_point
            if ll==0:
                p_cost=round(hold_cost+winp,1)
                index='Hold'+str(Hqty)
                trade.loc[index,str(i)+'W']=p_cost
            else:
                winp=round(winp,1)
                index='Lots'+str(ll)
                trade.loc[index,str(i)+'W']=winp

res

Get_url https://futures.htisec.com/wf/enquiryOrderList.do 200,Re:[]
Get_url https://futures.htisec.com/wf/enquiryOrderDetail.do?acc_order_no=2604&has_TS_falg=Y 200,Re:[]
Get_url https://futures.htisec.com/wf/enquiryOrderDetail.do?acc_order_no=2605&has_TS_falg=Y 200,Re:[]
Get_url https://futures.htisec.com/wf/enquiryOrderDetail.do?acc_order_no=2607&has_TS_falg=Y 200,Re:[]
Get_url https://futures.htisec.com/wf/enquiryOrderDetail.do?acc_order_no=2608&has_TS_falg=Y 200,Re:[]
Get_url https://futures.htisec.com/wf/enquiryOrderDetail.do?acc_order_no=2610&has_TS_falg=Y 200,Re:[]
Get_url https://futures.htisec.com/wf/enquiryOrderDetail.do?acc_order_no=2613&has_TS_falg=Y 200,Re:[]
{'HSIJ8': 0}


,Bqty,Sqty,Hqty,Sum,Cost,Charges,PointT,Point_NET,Point_AVG,Profit,Profit_NET,Profit_AVG
hold,0.0,0.0,0.0,0.0,0.0,0.00,1.3,NaN,NaN,NaN,NaN,NaN
close,34.0,-34.0,34.0,-457.0,NaN,2280.72,457.0,412.8,12.1,22850.0,20569.28,604.98
total,34.0,-34.0,0.0,-457.0,0.0,2280.72,0.0,-1.3,NaN,NaN,NaN,NaN


In [24]:
hh=web1.get_orderlist()
hh.loc[hh.refno==8730,'status']='Mark'


Get_url https://futures.htisec.com/wf/enquiryOrderList.do 200,Re:[]
